In [1]:
import os
import glob
import argparse
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
np.random.seed(111)

In [2]:
# Convert to dataframe
def convert_xml_to_df(files):
    data = []
    # Iterate over each file
    for sample in files:
        # Get the xml tree
        tree = ET.parse(sample)

        # Get the root
        root = tree.getroot()

        # Get the members and extract the values
        for member in root.findall('object'):
            filename = root.find('filename').text
            width =  int((root.find('size')).find('width').text)
            height = int((root.find('size')).find('height').text)
            label =  member.find('name').text

            bndbox = member.find('bndbox')
            xmin = float(bndbox.find('xmin').text)
            xmax = float(bndbox.find('xmax').text)
            ymin = float(bndbox.find('ymin').text)
            ymax = float(bndbox.find('ymax').text)

            data.append((filename, width, height, label, xmin, ymin, xmax, ymax))
    # Create a pandas dataframe
    columns_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    df = pd.DataFrame(data=data, columns=columns_name)

    return df

In [3]:
images = sorted(glob.glob('./felina/data_300x300/images/*.jpg'))
xmls = sorted(glob.glob('./felina/data_300x300/labels/*.xml'))
print("Total number of images: ", len(images))
print("Total number of xmls: ", len(xmls))

Total number of images:  1086
Total number of xmls:  1086


In [4]:
df = convert_xml_to_df(xmls)

In [5]:
df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,000045.jpg,300,300,aggregate reticulocyte,140.0,115.0,169.0,143.0
1,000045.jpg,300,300,punctate reticulocyte,72.0,155.0,103.0,187.0
2,000045.jpg,300,300,erythrocyte,184.0,195.0,213.0,228.0
3,000058.jpg,300,300,erythrocyte,174.0,65.0,203.0,94.0
4,000058.jpg,300,300,aggregate reticulocyte,120.0,113.0,151.0,143.0


In [6]:
df['class'].value_counts()

erythrocyte               1982
punctate reticulocyte     1007
aggregate reticulocyte    1001
Name: class, dtype: int64

In [7]:
train, valid = train_test_split(df, test_size=0.2, stratify=df['class'], random_state=111)

In [8]:
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
print("Number of training samples: ", len(train))
print("Number of validation samples: ", len(valid))

Number of training samples:  3192
Number of validation samples:  798


In [11]:
del df

---

## Dataframe to TFRecords

In [10]:
import sys
sys.path.append("..")
import io
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mimg
from PIL import Image
from collections import namedtuple, OrderedDict
from models.research.object_detection.utils import dataset_util
from models.research.object_detection.utils import label_map_util

In [16]:
# Group by imagefile name
def make_groups(df, field=None):
    if field==None:
        field = 'filename'
        
    data = namedtuple('object', ['filename', 'info'])
    grouped = df.groupby(field)
    
    grouped_data = []
    for filename, x in zip(grouped.groups.keys(), grouped.groups):
        grouped_data.append(data(filename, grouped.get_group(x)))
        
    return grouped_data    

In [13]:
def create_tf_example(group, img_path, label_map_dict):
    # Read the imagefile. This will be used in features later 
    with tf.gfile.GFile(os.path.join(img_path, '{}'.format(group.filename)), 'rb') as f:
        img_file = f.read()
    
    # Encode to bytes and read using PIL. Could be done directly too
    encoded_img = io.BytesIO(img_file)
    # Read the image using PIL
    img = Image.open(encoded_img)
    width, height = img.size
    
    # Encode the name of the img file
    filename = group.filename.encode('utf8')
    #print("Reading the file: ", group.filename)
    # Define the format of the image file
    img_format = b'jpg'   # The name will be in bytes
    
    
    # Define the variables that you need as features
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    # Iterate over the namedtuple object
    for index, row in group.info.iterrows():
        xmins.append(row['xmin'] / width)   # We have to store normalized values for bbox as per the TF api
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(label_map_dict[row['class']])

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(img_file),
        'image/format': dataset_util.bytes_feature(img_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),}))
    
    return tf_example

In [17]:
writer = tf.python_io.TFRecordWriter('./data/train.record') # The output path shouldn't be a directory

# Path where all the images are present
# img_path = './all_data_training/sampled_data/images/'
img_path = './felina/data_300x300/images/'

# create groups in the csvfile. One image may contain several instances of an object hence grouping
img_groups = make_groups(train, field='filename')

# Label map
label_map_dict = label_map_util.get_label_map_dict('./bloodmap.pbtxt')

# Iterate over the samples in each group create a TFRecord
for group in img_groups:
    tf_example = create_tf_example(group, img_path, label_map_dict)
    writer.write(tf_example.SerializeToString())

# close the writer
writer.close()
print("TFRecords created successfully")

TFRecords created successfully


In [18]:
writer = tf.python_io.TFRecordWriter('./data/valid.record') # The output path shouldn't be a directory

# Path where all the images are present
# img_path = './all_data_training/sampled_data/images/'
img_path = './felina/data_300x300/images/'

# create groups in the csvfile. One image may contain several instances of an object hence grouping
img_groups = make_groups(valid, field='filename')

# Label map
label_map_dict = label_map_util.get_label_map_dict('./bloodmap.pbtxt')

# Iterate over the samples in each group create a TFRecord
for group in img_groups:
    tf_example = create_tf_example(group, img_path, label_map_dict)
    writer.write(tf_example.SerializeToString())

# close the writer
writer.close()
print("TFRecords created successfully")

TFRecords created successfully
